# Data Preprocessing : Download Used Cars Dataset 
https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes

In [ ]:
# change here with the directory where you downloaded the dataset
data_dir = './data'

In [ ]:
import pandas as pd

# Dataset Details
The data consists of used cars listings. 100,000 listings, which have been separated into files corresponding to each car manufacturer. Each file will simulate data for each node.

# Goal

The goal of this tutorial is to build a federated regression model on Non-IID dataset and generate the best model by performing validation on hold out dataset and tuning hyperparameters.The metric used to decide best model is RMSE.

In [ ]:
import os

# Use audi and bmw for training on 2 nodes
audi = pd.read_csv(os.path.join(data_dir, "audi.csv"))
bmw = pd.read_csv(os.path.join(data_dir, "bmw.csv"))

# Use Ford for final validation at central researcher (test dataset)
ford = pd.read_csv(os.path.join(data_dir, "ford.csv"))

# Use the following csvs if you want to run more than 2 nodes. Uncomment Corresponding lines in the following cell blocks
# cclass = pd.read_csv(os.path.join(data_dir, "cclass.csv"))
# focus = pd.read_csv(os.path.join(data_dir, "focus.csv"))
# hyundai = pd.read_csv(os.path.join("data_dir, "huyndai.csv"))
# merc = pd.read_csv(os.path.join(data_dir, "merc.csv"))
# skoda = pd.read_csv(os.path.join(data_dir, "skoda.csv"))
# toyota = pd.read_csv(os.path.join(data_dir, "toyota.csv"))
# vauxhall = pd.read_csv(os.path.join(data_dir, "vauxhall.csv"))
# vw = pd.read_csv(os.path.join(data_dir, "vw.csv"))

Drop columns model & fuelType as labels are not consistent across files. A better solution could be vertical federated learning

In [ ]:
audi.drop(columns = ['model','fuelType'],inplace = True)
bmw.drop(columns = ['model','fuelType'],inplace = True)
ford.drop(columns = ['model','fuelType'],inplace = True)

# cclass.drop(columns = ['model','fuelType'],inplace = True)
# focus.drop(columns = ['model','fuelType'],inplace = True)
# hyundai.drop(columns = ['model','fuelType'],inplace = True)
# merc.drop(columns = ['model','fuelType'],inplace = True)
# skoda.drop(columns = ['model','fuelType'],inplace = True)
# toyata.drop(columns = ['model','fuelType'],inplace = True)
# vauxhall.drop(columns = ['model','fuelType'],inplace = True)
# vw.drop(columns = ['model','fuelType'],inplace = True)

Label encode transmission column

In [ ]:
audi['transmission'] = audi['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
bmw['transmission'] = bmw['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
ford['transmission'] = ford['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})

# cclass['transmission'] = cclass['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# focus['transmission'] = focus['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# hyundai['transmission'] = hyundai['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# merc['transmission'] = merc['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# skoda['transmission'] = skoda['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# toyata['transmission'] = toyata['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# vauxhall['transmission'] = vauxhall['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# vw['transmission'] = vw['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})

In [ ]:
audi.to_csv(os.path.join(data_dir, 'audi_transformed.csv'),header = True,index= False)
bmw.to_csv(os.path.join(data_dir, 'bmw_transformed.csv'),header = True,index= False)
ford.to_csv(os.path.join(data_dir, 'ford_transformed.csv'),header = True,index= False)

# cclass.to_csv('cclass_transformed.csv',header = True,index= False)
# focus.to_csv('focus_transformed.csv',header = True,index= False)
# hyundai.to_csv('huydai_transformed.csv',header = True,index= False)
# merc.to_csv('merc_transformed.csv',header = True,index= False)
# skoda.to_csv('skoda_transformed.csv',header = True,index= False)
# toyata.to_csv('toyata_transformed.csv',header = True,index= False)
# vauxhall.to_csv('vaxhall_transformed.csv',header = True,index= False)
# vw.to_csv('vw_transformed.csv',header = True,index= False)

# Fed-BioMed Researcher to train a model on a Used Cars dataset

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the nodes up
It is necessary to previously configure 2 nodes:
1. `./scripts/fedbiomed_run node config config1.ini add` (node1) and `./scripts/fedbiomed_run node config config2.ini add` (node2)
  * Select option 1 to add a csv file to the node
    * use the `audi_transformed.csv` file (node 1) and `bmw_transformed.csv` file (node 2)
  * Choose the name, tags and description of the dataset
    * choose tag `UsedCars` (or modify the used tag in this notebook)
  * Spin as many nodes as you want(max nodes 11 for 11 csv files in used cars dataset). Hold out one file for testing.
  * Load the .csv file generated using above mentioned notebook to individual nodes
2. Check that your data has been added by executing `./scripts/fedbiomed_run node config config1.ini list` (node1) and `./scripts/fedbiomed_run node config config2.ini list` (node2)
3. Run the node using `./scripts/fedbiomed_run node config config1.ini start` (node1) and `./scripts/fedbiomed_run node config config2.ini start` (node2). Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager 


# Here we define the model to be used. 
# You can use any class name (here 'MyTrainingPlan')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        # should match the model arguments passed below to the experiment class
        self.in_features = model_args['in_features']
        self.out_features = model_args['out_features']
        self.fc1 = nn.Linear(self.in_features, 5)
        self.fc2 = nn.Linear(5, self.out_features)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch Dataset and DataLoader classes
        # We need pandas to read the local .csv file at the node side
        deps = ["from torch.utils.data import Dataset",
                "import pandas as pd"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, data, target):
        output = self.forward(data).float()
        criterion = torch.nn.MSELoss()
        loss   = torch.sqrt(criterion(output, target.unsqueeze(1)))
        return loss

    class csv_Dataset(Dataset):
    # Here we define a custom Dataset class inherited from the general torch Dataset class
    # This class takes as argument a .csv file path and creates a torch Dataset 
        def __init__(self, dataset_path, x_dim):
            self.input_file = pd.read_csv(dataset_path,sep=',',index_col=False)
            x_train = self.input_file.loc[:,('year','transmission','mileage','tax','mpg','engineSize')].values
            y_train = self.input_file.loc[:,'price'].values
            self.X_train = torch.from_numpy(x_train).float()
            self.Y_train = torch.from_numpy(y_train).float()

        def __len__(self):            
            return len(self.Y_train)

        def __getitem__(self, idx):

            return (self.X_train[idx], self.Y_train[idx])
        
    def training_data(self,  batch_size = 48):
    # The training_data creates the Dataloader to be used for training in the general class TorchTrainingPlan of fedbiomed
        dataset = self.csv_Dataset(self.dataset_path, self.in_features)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset , **train_kwargs)
    
    

In [11]:
# model parameters 
model_args = {
    'in_features': 6, 
    'out_features': 1
}

# training parameters 
training_args = {
    'batch_size': 40, 
    'lr': 1e-3, 
    'epochs': 2, 
    'dry_run': False,  
    #'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [12]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# Calling the training data with specified tags. Change the following tag accordingly
tags =  ['UsedCars']
rounds = 3

exp = Experiment(tags=tags,
                 model_class=MyTrainingPlan,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-08 15:26:07,170 fedbiomed INFO - Searching dataset with data tags: ['UsedCars'] for all nodes
2022-03-08 15:26:07,175 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['UsedCars'], 'command': 'search'}
2022-03-08 15:26:17,213 fedbiomed INFO - Node selected for training -> node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0
2022-03-08 15:26:17,214 fedbiomed INFO - Node selected for training -> node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0
2022-03-08 15:26:17,245 fedbiomed DEBUG - Model file has been saved: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0082/my_model_8181e794-f0bb-4402-8830-3b687aa03bf0.py
2022-03-08 15:26:17,286 fedbiomed DEBUG - upload (HTTP POST request) of file /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0082/my_model_8181e794-f0bb-4402-8830-3b687aa03bf0.py successful, with status code 20

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [13]:
exp.run()

2022-03-08 15:26:20,079 fedbiomed INFO - Sampled nodes in round 0 ['node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0']
2022-03-08 15:26:20,080 fedbiomed INFO - Send message to node node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 - {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': '09302378-1890-4ee6-a256-76027c15053e', 'training_args': {'batch_size': 40, 'lr': 0.001, 'epochs': 2, 'dry_run': False}, 'model_args': {'in_features': 6, 'out_features': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/08/my_model_8181e794-f0bb-4402-8830-3b687aa03bf0.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/08/aggregated_params_init_42bbfbff-04a9-4cb3-9f55-70627b36eb09.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0': ['dataset_5ed4224a-ae70-4661-b0ef-588f589522a3']}}
2022-03-08 15:26:20,082 fedbiomed DEBUG - researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab
2022-03-08 15:26:20,087 fedbiomed

2022-03-08 15:26:20,881 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [800/10668 (7%)]	Loss: 24811.699219
2022-03-08 15:26:20,905 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [1200/10668 (11%)]	Loss: 22539.281250
2022-03-08 15:26:20,924 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [1600/10668 (15%)]	Loss: 24391.896484
2022-03-08 15:26:20,944 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [2000/10668 (19%)]	Loss: 21823.052734
2022-03-08 15:26:20,966 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [2400/10668 (22%)]	Loss: 23376.527344
2022-03-08 15:26:20,985 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [2800/10668 (26%)]	Loss: 21228.833984
2022-03-08 15:26:21,001 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f

2022-03-08 15:26:30,187 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - upload (HTTP GET request) of file my_model_8c84562a-485c-4d22-bca3-4eb5ee45dcaa.pt successful, with status code 200
2022-03-08 15:26:30,195 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Dataset path has been set as/home/scansiz/Desktop/Inria/development/data/used-cars/processed/audi_transformed.csv
2022-03-08 15:26:30,224 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - training with arguments {'monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f14ab3b1a60>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'training_data': <torch.utils.data.dataloader.DataLoader object at 0x7f14ab387df0>, 'batch_size': 40, 'lr': 0.001, 'epochs': 2, 'dry_run': False}
2022-03-08 15:26:30,225 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Using device cpu for training (cuda_av

2022-03-08 15:26:31,343 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [8000/10668 (75%)]	Loss: 16316.448242
2022-03-08 15:26:31,372 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [8400/10668 (79%)]	Loss: 14288.474609
2022-03-08 15:26:31,393 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [8800/10668 (82%)]	Loss: 19652.939453
2022-03-08 15:26:31,419 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [9200/10668 (86%)]	Loss: 16544.816406
2022-03-08 15:26:31,450 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [9600/10668 (90%)]	Loss: 21827.767578
2022-03-08 15:26:31,485 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [10000/10668 (94%)]	Loss: 23401.089844
2022-03-08 15:26:31,532 fedbiomed INFO - log from: node_bd90cc6f-67c4

2022-03-08 15:26:40,691 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [4400/10668 (41%)]	Loss: 17369.023438
2022-03-08 15:26:40,722 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [4800/10668 (45%)]	Loss: 17254.078125
2022-03-08 15:26:40,739 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [5200/10668 (49%)]	Loss: 17631.453125
2022-03-08 15:26:40,760 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [5600/10668 (52%)]	Loss: 19142.195312
2022-03-08 15:26:40,776 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [6000/10668 (56%)]	Loss: 18628.037109
2022-03-08 15:26:40,795 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [6400/10668 (60%)]	Loss: 23292.607422
2022-03-08 15:26:40,814 fedbiomed INFO - log from: node_bd90cc6f-67c4-

...

2022-03-08 16:15:37,348 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': '31c8661b-48f8-4db6-a232-acb26c20652e', 'params_url': 'http://localhost:8844/media/uploads/2022/03/08/aggregated_params_b9416a3b-294c-4dc4-9f6f-b241579e70ed.pt', 'training_args': {'epochs': 5}, 'training_data': {'node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0': ['dataset_1e77184c-4217-4c38-a0be-14d43df40c21']}, 'model_args': {'model': 'Perceptron', 'max_iter': 1000, 'tol': 0.001, 'n_features': 20, 'n_classes': 2, 'verbose': 1}, 'model_url': 'http://localhost:8844/media/uploads/2022/03/08/my_model_b508dd3f-46ac-4b62-8f99-11566b0a5aa8.py', 'model_class': 'SkLearnTrainingPlan', 'command': 'train'}
2022-03-08 16:15:37,358 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - upload (HTTP GET request) of file my_model_77e25db7adbd4dc982dba571e153c527.py successful, with sta

2022-03-08 16:18:53,444 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': '099b3d60-decf-4523-81ba-96ffad80abfd', 'params_url': 'http://localhost:8844/media/uploads/2022/03/08/aggregated_params_init_4f42e690-b476-4d91-9288-8ef8b632f01d.pt', 'training_args': {'epochs': 5}, 'training_data': {'node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0': ['dataset_cc5be93f-bfbf-4343-be4e-85252407b135']}, 'model_args': {'max_iter': 2000, 'tol': 1e-05, 'eta0': 0.05, 'model': 'SGDRegressor', 'n_features': 8, 'random_state': 1234, 'verbose': 1}, 'model_url': 'http://localhost:8844/media/uploads/2022/03/08/my_model_16f8ec71-2bbf-4cd5-8187-80a6ff65b97e.py', 'model_class': 'SGDRegressorTrainingPlan', 'command': 'train'}
2022-03-08 16:18:53,453 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - upload (HTTP GET request) of file my_model_c5b0295afacd4225bdce75382

2022-03-08 16:26:19,962 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['adni'], 'command': 'search'}
2022-03-08 16:26:30,123 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': '407669c9-93a1-4a1a-ab00-4468a338f5a1', 'training_args': {'epochs': 5}, 'model_args': {'max_iter': 2000, 'tol': 1e-05, 'eta0': 0.05, 'model': 'SGDRegressor', 'n_features': 8, 'random_state': 1234, 'verbose': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/08/my_model_463f0b23-03b9-4b64-bc1e-6f56a4d97f26.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/08/aggregated_params_init_76e2735a-f45e-443f-8cb1-81fd4b3d32ef.pt', 'model_class': 'SGDRegressorTrainingPlan', 'training_data': {'node_bd90cc6f-67c4-4f81-aaa5-588c9c26c

2022-03-08 16:33:10,600 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / ERROR - Did not found proper data in local datasets on node=node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0
2022-03-08 16:36:39,866 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / CRITICAL - Node stopped in signal_handler, probably by user decision (Ctrl C)
2022-03-08 16:36:40,054 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / CRITICAL - Node stopped in signal_handler, probably by user decision (Ctrl C)
2022-03-08 16:37:28,607 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Starting task manager
2022-03-08 16:37:33,803 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['#test_data'], 'command': 'search'}
2022-03-08 16:38:10,830 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEB

2022-03-08 16:38:21,609 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 6 [800/1000 (80%)]	Loss: 382.471924
2022-03-08 16:38:21,622 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [0/1000 (0%)]	Loss: 233.559036
2022-03-08 16:38:21,632 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [200/1000 (20%)]	Loss: 155.664841
2022-03-08 16:38:21,644 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [400/1000 (40%)]	Loss: 336.358185
2022-03-08 16:38:21,655 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [600/1000 (60%)]	Loss: 374.487854
2022-03-08 16:38:21,665 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [800/1000 (80%)]	Loss: 365.745087
2022-03-08 16:38:21,675 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / IN

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


# Test Function

In [ ]:
fed_model = exp.model_instance()
fed_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

In [ ]:
 fed_model

In [ ]:
# Hold one file for testing the fed model
test_dataset_path = os.path.join(data_dir, "ford_transformed.csv")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def cal_rmse(actual, prediction):
    return ((actual- prediction)**2).mean()**0.5

def testing_rmse(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'
    preds = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds.append(output.numpy().flatten())
    rmse = cal_rmse(data_loader.dataset.Y_train.numpy(),np.hstack(preds))
    return rmse

In [ ]:
class csv_Dataset(Dataset):
        def __init__(self, dataset_path):
            self.input_file = pd.read_csv(dataset_path,sep=',',index_col=False)
            x_train = self.input_file.loc[:,('year','transmission','mileage','tax','mpg','engineSize')].values
            y_train = self.input_file.loc[:,'price'].values
            self.X_train = torch.from_numpy(x_train).float()
            self.Y_train = torch.from_numpy(y_train).float()

        def __len__(self):            
            return len(self.Y_train)

        def __getitem__(self, idx):

            return (self.X_train[idx], self.Y_train[idx])

In [ ]:
dataset = csv_Dataset(test_dataset_path)
train_kwargs = {'batch_size': 64, 'shuffle': True}
data_loader = DataLoader(dataset, **train_kwargs)

In [ ]:
rmse = testing_rmse(fed_model, data_loader)

In [ ]:
rmse